In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from functools import reduce
from scipy.stats import linregress
from scipy import fftpack
from scipy.signal import detrend
from scipy.optimize import root_scalar
from tqdm import tqdm
from scipy import special
from statsmodels.tsa.stattools import acf, pacf
from tqdm.notebook import tqdm_notebook

In [2]:
'''
Cell size evolution x(t)
'''
def x_function(t, u, xb, omega1 = 1):
    

    x = (xb+u)*np.exp(omega1*t)-u
    return x
    

'''
Hazard rate function h(t)
'''
def h(t, u, xb, omega2, omega1 = 1, v = 1):
    

    h = omega2*(1+x_function(t, u, xb, omega1)/v)
    return h    


'''
Logarithm of survival function s(t)
'''
def SDF(t, omega2, u, xb, omega1 = 1, v = 1):
    
    s = np.exp(omega2*t*(u/v - 1) + (omega2/omega1)*((u + xb)/v)*(1-np.exp(omega1*t)))
    return s


In [3]:
def sizesAtBirth(omega2, omega1 = 1, xb0 = 1, u = 0, v = 1, seriesLength = 10000):
    pointsPerEvolution = 200 # Number of points per cell division
    sizesAtBirth = np.zeros(seriesLength)
    #timesAtBirth = np.zeros(seriesLength)
    
    
    s_drawn = np.random.uniform(low=0, high = 1, size = seriesLength)
    # Initialize the cell size array
    cellSizes = np.zeros(pointsPerEvolution*seriesLength)
    xb = xb0
    t = 0
    for i in range(seriesLength): 
        sizesAtBirth[i] = xb
        #parameters = (1,0.9,0.1,1,mb) # omega1, omega2, mu, nu, xb
        sol = root_scalar(lambda t: SDF(t, omega2, u, xb) - s_drawn[i], bracket=[0, 40], method='brentq')
        tau = sol.root
        #sim_t.append(tau)

        # evolution
        times = np.linspace(0, tau, pointsPerEvolution)
        xt = x_function(times, u = u, xb = xb, omega1 = omega1)
        # pt = p_function(times, parameters)
        
        # store times and sizes
        #all_times[i*points_per_evolution : (i+1)*points_per_evolution] = np.linspace(t, t+tau, points_per_evolution)
        cellSizes[i*pointsPerEvolution : (i+1)*pointsPerEvolution] = xt

        # update the initial time and the starting size
        xb = xt[-1]*(1/2)
        t = t+tau
    
    return sizesAtBirth, cellSizes


In [4]:
criticalPoint = 1 / np.log(2)
omega2Values = - np.abs(np.logspace(np.log10(np.abs(1 - criticalPoint)), np.log10(1e-3), 100))  + criticalPoint
gamma_gammaC = np.abs(np.logspace(np.log10(np.abs(1 - criticalPoint)), np.log10(1e-3), 100))

In [5]:
def cumulative(b, delta, m):
    return b**(-delta) * m**(delta)

def inverseCum(b, delta, u):
    return b * u**(1/delta)

def delta(distance):
    return 0.76*distance

In [19]:
seriesLength = 1000000
simulate = True
absorbingValues = []
if simulate:
    startingPointsUniform = np.random.uniform(0, 1, len(omega2Values))
    startingPoints = [inverseCum(b = 10, delta = delta(gamma_gammaC[i]), u = startingPointsUniform[i]) for i in range(len(omega2Values))]
    aCorrValues = np.zeros((len(omega2Values), seriesLength ))
    #allSizes = np.zeros((len(omega2Values), 100000))
    for i, omega2 in enumerate(tqdm_notebook(omega2Values)):
        #sizes = np.zeros(100000)
        sizeBirth, _ = sizesAtBirth(omega2 = omega2, omega1 = 1, xb0 = startingPoints[i], u = 0, v = 1, seriesLength = seriesLength)
        #sizes += sizeBirth[-100000:]
        #allSizes[i,:] = sizes
        autocorrelation = acf(sizeBirth, fft = True, nlags = seriesLength - 1)
        aCorrValues[i, :] += autocorrelation #]/ nTrajectoires 
        

            
    #np.save("../../../data/sizeAtBirthShort1Milion.npy", allSizes)
    np.save("../../../data/aCorrValuesShort1MilionSupplementary.npy", aCorrValues)

  0%|          | 0/16 [00:00<?, ?it/s]

In [16]:
acf(sizeBirth, fft = True, nlags = seriesLength - 1)

array([ 1.00000000e+00,  8.23115143e-01,  7.01182500e-01, ...,
       -2.94371130e-05, -2.44404808e-05, -1.51863074e-05])